# Seattle Building Energy Forecasting

URL: https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking

## Notebook n°5 - Modelling

Objective: Create dummy variables and try different models

---

In [63]:
import time
import warnings
import re
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

## Settings

In [64]:
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 200

target1 = "SiteEnergyUse(kBtu)"
target2 = "TotalGHGEmissions"

TARGET = target2

## Import data

In [65]:
with open("data/part4.pkl", "rb") as f:
    my_unpickler = pickle.Unpickler(f)
    data = my_unpickler.load()
    
with open("data/part4-data-with-outliers.pkl", "rb") as f:
    my_unpickler = pickle.Unpickler(f)
    data_with_outliers = my_unpickler.load()

In [66]:
data.head()

,DataYear,BuildingType,PrimaryPropertyType,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofFloors,LargestPropertyUseType,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions,Latitude,Longitude,Address,ZipCode,default_data,lat_long_range,address_type
0,2015,NonResidential,Hotel,7,DOWNTOWN,1927,12.0,Hotel,65.0,6981428.0,249.43,47.612190,-122.337997,405 OLIVE WAY,98101.0,False,62,Way
1,2015,NonResidential,Hotel,7,DOWNTOWN,1996,11.0,Hotel,51.0,8354235.0,263.51,47.613106,-122.333358,724 PINE ST,98101.0,False,62,Street
5,2015,Nonresidential COS,Other,7,DOWNTOWN,1999,2.0,Police Station,NaN,12051984.0,304.62,47.616439,-122.336764,810 VIRGINIA ST,98101.0,False,62,Street
6,2015,NonResidential,Hotel,7,DOWNTOWN,1926,11.0,Hotel,25.0,6252842.0,208.46,47.614114,-122.332741,1619 9TH AVE,98101.0,False,62,Avenue
7,2015,NonResidential,Other,7,DOWNTOWN,1926,8.0,Other - Entertainment/Public Assembly,NaN,6426022.0,199.99,47.612902,-122.331309,901 PINE ST,98101.0,False,62,Street


## Functions

In [67]:
def onehot(data, feature):
    
    # Get dummy variables
    temp_df = pd.get_dummies(data[feature])
    
    # Add prefix to prevent duplicated feature names
    temp_df = temp_df.add_prefix(feature + "_")
    
    # Concatenante the new features with the main dataframes
    data = pd.concat([data, temp_df], axis=1)
    
    # Drop the original feature
    data.drop(feature, axis=1, inplace=True)
    
    # Return the new dataframe
    return data

def cat_analysis(df, feature, x, y, figsize=(8,5), rotation="45", palette=None, order=None):
    
    data = (pd.DataFrame(df[feature].value_counts())
                .reset_index()
                .rename(columns={'index': x, feature: y}))
    
    fig, ax = plt.subplots(figsize=figsize)
    ax = sns.barplot(x=x, y=y, data=data, palette=palette, order=order);
    plt.xticks(rotation=rotation)
    plt.show()
    
def prepare_data(data, energystarscore="fill"):
    
    # Copy original data
    data_copy = data.copy()
    
    # Building Type
    data_copy = onehot(data_copy, "BuildingType")
    
    # CouncilDistrictCode
    data_copy = onehot(data_copy, "CouncilDistrictCode")
    
    # Neighborhood
    data_copy = onehot(data_copy, "Neighborhood")
    
    # LargestPropertyUseType
    data_copy = onehot(data_copy, "LargestPropertyUseType")
    
    # PrimaryPropertyType
    data_copy = onehot(data_copy, "PrimaryPropertyType")
    
    # DataYear
    data_copy.drop("DataYear", axis=1, inplace=True)
    
    # Address
    data_copy.drop("Address", axis=1, inplace=True)
    
    # address_type
    data_copy.drop("address_type", axis=1, inplace=True)
    
    # lat_long_range
    data_copy.drop("lat_long_range", axis=1, inplace=True)
    
    # ZipCode
    data_copy.drop("ZipCode", axis=1, inplace=True)
    
    # default_data
    data_copy.drop("default_data", axis=1, inplace=True)
    
    # ENERGYSTARScore
    if (energystarscore == "fill"):
        data_copy["ENERGYSTARScore"].fillna(data_copy["ENERGYSTARScore"].dropna().mean(), inplace=True)
    elif (energystarscore == "drop"):
        mask = data_copy["ENERGYSTARScore"].isna()
        data_copy = data_copy[~mask]

    return data_copy

In [76]:
data_prep = prepare_data(data)
data_prep_ess = prepare_data(data, energystarscore="drop")
data_with_outliers_prep = prepare_data(data_with_outliers, energystarscore="drop")

## Overview

In [69]:
data.head()

,DataYear,BuildingType,PrimaryPropertyType,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofFloors,LargestPropertyUseType,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions,Latitude,Longitude,Address,ZipCode,default_data,lat_long_range,address_type
0,2015,NonResidential,Hotel,7,DOWNTOWN,1927,12.0,Hotel,65.0,6981428.0,249.43,47.612190,-122.337997,405 OLIVE WAY,98101.0,False,62,Way
1,2015,NonResidential,Hotel,7,DOWNTOWN,1996,11.0,Hotel,51.0,8354235.0,263.51,47.613106,-122.333358,724 PINE ST,98101.0,False,62,Street
5,2015,Nonresidential COS,Other,7,DOWNTOWN,1999,2.0,Police Station,NaN,12051984.0,304.62,47.616439,-122.336764,810 VIRGINIA ST,98101.0,False,62,Street
6,2015,NonResidential,Hotel,7,DOWNTOWN,1926,11.0,Hotel,25.0,6252842.0,208.46,47.614114,-122.332741,1619 9TH AVE,98101.0,False,62,Avenue
7,2015,NonResidential,Other,7,DOWNTOWN,1926,8.0,Other - Entertainment/Public Assembly,NaN,6426022.0,199.99,47.612902,-122.331309,901 PINE ST,98101.0,False,62,Street


## Random Forest Model

In [99]:
def my_model(data, target, param_grid, test_size, target_log=False):

    X = data.drop([target1, target2], axis=1)
    y = data[target]
    
    if target_log:
        y = np.log(data[target])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    score = 'neg_mean_squared_error'

    # Cross validation classifier to get best parameters from the grid search
    clf_rf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring=score)
    clf_rf.fit(X_train_scaled, y_train)
    params = clf_rf.best_params_
        
    # Train prediction
    y_train_pred = clf_rf.predict(X_train_scaled)
    
    # Test prediction
    y_test_pred = clf_rf.predict(X_test_scaled)
    
    ############## Scores ##############
    
    # Get back to exponantial if we are in log for the target
    if target_log:
        y_train_pred = np.exp(y_train_pred)
        y_train = np.exp(y_train)
        y_test_pred = np.exp(y_test_pred)
        y_test = np.exp(y_test)
    
    # Compute R2 and RMSE for both training set and test set
    train_rmse = np.sqrt(mean_squared_error(y_train_pred, y_train))
    train_r2 = r2_score(y_train_pred, y_train)
    test_rmse = np.sqrt(mean_squared_error(y_test_pred, y_test))
    test_r2 = r2_score(y_test_pred, y_test)
    
    # Push the score in a dict
    scores = {
        'train_rmse': train_rmse,
        'train_r2': train_r2,
        'test_rmse': test_rmse,
        'test_r2': test_r2
    }
    
    # Return the classifier, y test and scores
    return clf_rf, y_test_pred, y_test, scores

## Run it - Site Energy Use

#### Data without EnergyStarScore

In [89]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }

classifier1, y_pred, y_test, scores = my_model(data_prep.drop("ENERGYSTARScore", axis=1), target1, param_grid, test_size=0.20)
display(classifier1.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 659444.4432438054,
 'train_r2': 0.9615378919272551,
 'test_rmse': 1923906.5763574117,
 'test_r2': 0.601029734411783}

#### Data with EnergyStarScore + filled empty values

In [74]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }

classifier2, y_pred, y_test, scores = my_model(data_prep, target1, param_grid, test_size=0.20)
display(classifier1.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 666097.7116605289,
 'train_r2': 0.9608423871375331,
 'test_rmse': 1896538.2733693456,
 'test_r2': 0.6178497238074636}

#### Data with only EnergyStarScore available

In [93]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }
classifier3, y_pred, y_test, scores = my_model(data_prep_ess, target1, param_grid, test_size=0.20)
display(classifier3.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 694553.2281850692,
 'train_r2': 0.9572652192404036,
 'test_rmse': 1676413.3050465945,
 'test_r2': 0.743560061515294}

#### Data with outliers

In [78]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }

classifier4, y_pred, y_test, scores = my_model(data_with_outliers_prep, target1, param_grid, test_size=0.2)
display(classifier4.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 4215208.219864903,
 'train_r2': 0.8860165395613738,
 'test_rmse': 7594725.810338778,
 'test_r2': 0.6383984615778551}

## Run it - TotalGHGEmissions

#### Data without EnergyStarScore

In [100]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }

classifier5, y_pred, y_test, scores = my_model(data_prep.drop("ENERGYSTARScore", axis=1), target2, param_grid, test_size=0.20)
display(classifier5.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 16.514009407217518,
 'train_r2': 0.9376383982980593,
 'test_rmse': 47.12160124033845,
 'test_r2': 0.30869854674751784}

#### Data with EnergyStarScore + filled empty values

In [101]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }

classifier6, y_pred, y_test, scores = my_model(data_prep, target2, param_grid, test_size=0.20)
display(classifier6.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 16.814221487571963,
 'train_r2': 0.9343754119423607,
 'test_rmse': 47.02744383763117,
 'test_r2': 0.2556774440738857}

#### Data with only EnergyStarScore available

In [102]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }
classifier7, y_pred, y_test, scores = my_model(data_prep_ess, target2, param_grid, test_size=0.20)
display(classifier7.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 16.83488228865185,
 'train_r2': 0.9332441267183788,
 'test_rmse': 44.77393721536224,
 'test_r2': 0.32742952247137}

#### Data with outliers

In [103]:
param_grid = {'n_estimators': [100],
              'max_features': ["auto"],
              'n_jobs': [-1],
              'random_state': [42]
              }

classifier8, y_pred, y_test, scores = my_model(data_with_outliers_prep, target2, param_grid, test_size=0.2)
display(classifier8.best_params_)
display(scores)

{'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'random_state': 42}

{'train_rmse': 143.30817549237543,
 'train_r2': 0.8612698286776959,
 'test_rmse': 291.624469542639,
 'test_r2': 0.40599412721476014}